In [ ]:
#import required libraries
import matplotlib
from matplotlib import pyplot as plt
import numpy as np
import pandas as pd
import skbio
import skbio.diversity
from skbio.diversity import * #pw_distances was renamed to beta_diversity and was moved to skbio.diversity
from skbio.stats.distance import mantel
from skbio.stats.ordination import *
from mpl_toolkits.mplot3d import Axes3D
import seaborn as sns

In [ ]:
#import the OTU table and metadata table

OTUtable=pd.read_csv("fullabsOTUsl7copy.csv",index_col='#OTU ID')
metadata=pd.read_csv("mapflcategorical.tsv",delimiter="\t",index_col='#Sample ID')

Let's first go through our route OTU and metadata table cleaning workflow, as in Python Lesson 8 (Pandas Part2)

In [ ]:
#show chloroplast entries in OTU table
OTUtable[OTUtable.index.str.contains('Chloroplast|chloroplast')]

In [ ]:
#filter chloroplasts out of OTU table
OTUtable=OTUtable[~OTUtable.index.str.contains('Chloroplast|chloroplast')]

In [ ]:
#show rows from OTU table that have less than 10 reads in all samples
OTUtable.loc[(OTUtable<10).all(axis=1)]

In [ ]:
#remove rows from OTU table that have less than 10 reads in all samples
OTUtable=OTUtable.loc[~(OTUtable<10).all(axis=1)]

In [ ]:
OTUtable

In [ ]:
#filter the metadata table. Need to drop any samples that are not in the OTU table (failed sequencing, decided not to sequence, etc)
#samples that were successfully sequenced correspond to OTUtable column headers
samplesSequenced=OTUtable.columns

In [ ]:
toRem=[]
for i in metadata.index:
    if i not in samplesSequenced:
        print(i,'not sequenced')
        toRem.append(i)

In [ ]:
#show the rows in metadata table that were not successfully sequenced
metadata.loc[toRem]

In [ ]:
#drop from metadata
clMetadata=metadata.drop(toRem)

The OTU pandas DataFrame has to be manipulated into the appropriate format for the Bray-Curtis and Jaccard Distance matrices. We have to define our sample IDs, meaning the discrete samples (not OTU names), swap the rows and columns of our OTUtable so that OTU IDs are columns and discrete samples are rows, and finally, convert the modified OTUtable to a fully numeric array.

In [ ]:
#set up the OTU data for Bray-Curtis and Jaccard Distance matrices- requires separating sample IDs
#and formatting the numerical data as an array

sampleIDs=OTUtable.columns #matrix IDs are sample IDs, not OTU ids

In [ ]:
sampleIDs

In [ ]:
#need to transpose rows and columns such that the sampleIDs become index and the OTU ids become column headers
OTUdata=OTUtable.T.values

In [ ]:
OTUdata

Now that our OTU data is in the correct format, we can use the scikitbio `beta_diversity` function to create our jaccard and bray curtis distance matrices. The function arguments include the type of analysis to be performed and the appropriate input data. Specifics for different analyses can be found in the scikitbio documentation: http://scikit-bio.org/docs/latest/index.html

In [ ]:
#jaccard distance is unweighted (doesnt consider abundance/phylogeny)- just tells us whether or not features are shared
j_dm = beta_diversity("jaccard",OTUdata, sampleIDs)

Conveniently, the scikit bio jaccard and bray curtis distance matrices are automatically in a format such that typing them in will show a heatmap.

In [ ]:
j_dm

In [ ]:
#bray curtis is similar to jaccard but weighted
bc_dm = beta_diversity("braycurtis",OTUdata, sampleIDs)

In [ ]:
bc_dm

A mantel test is a common "checkpoint" analysis that is used to compare bray-curtis and jaccard distance matrices. Usually this is to make sure that you have no accidentally used a binary array for bray-curtis and actual abundance data for jaccard and vice-versa. This is avoided by using scikitbio because you do not have to convert the OTU data to a binary array first. The function does this internally. We still expect that there is some correlation between the two. we will re-evaluate this later by plotting the difference between the two matrices as a heatmap.

In [ ]:
#can determine if jaccard and bray-curtis matrices are significantly correlated with one another using Mantel correlation
r, p_value, n = mantel(j_dm, bc_dm)

In [ ]:
r,p_value,n

Once the distance matrices have been created, we can use them to perform a principle component ordination analysis. 

In [ ]:
#perform PCoA on both distance matrices, show the proportions explained by each PC
bc_PCoA = pcoa(bc_dm)
bc_PCoA_prps=bc_PCoA.proportion_explained
bc_PCoA_mat=bc_PCoA.samples

In [ ]:
bc_PCoA_mat

In [ ]:
j_PCoA = pcoa(j_dm)
j_PCoA_prps=j_PCoA.proportion_explained
j_PCoA_mat=j_PCoA.samples

In [ ]:
j_PCoA_mat

In [ ]:
#Visualization- look at pcoa result matrices in seaborn heatmap, then look at 3D plots of the matrices color-coded by metadata
#seaborn
#examine all 3 together, then close

sns.heatmap(bc_PCoA.samples)

In [ ]:
sns.heatmap(j_PCoA.samples)

We will see below that although the two matrices are infact correlated, their is still a visible difference between the two.

In [ ]:
sns.heatmap(bc_PCoA.samples - j_PCoA.samples)

In [ ]:
plt.close()

In [ ]:
#3D scatter with metadata
#before we can do this, we need to remove any metadatacolumns that have NaN, or it won't work
#pandas has a great function that returns columns with NaN!

boolColNaN=clMetadata.isna().any()

#get names of columns that don't have NaN, use to make dataframe
noNanCols=boolColNaN[boolColNaN==False].index

clMetadataNoNaN=clMetadata[noNanCols]

In [ ]:
#don't x out plots until after both- show differences in the clustering using the two types of distance matrices

Carefully compare some of the Jaccard and Bray-Curtis PCoA plots that are color-coded with categorical chemical data (e.g., methane, sulfide). The Jaccard clustering patterns will tell us more about how these variables drives the presence/absence of OTUs, while the Bray-Curtis PCoA will tell us more about how the relative abundances of those OTUs are driven by the chemistry.

In [ ]:
#bray curtis PCoA
for col in clMetadataNoNaN.columns:
    if col != 'Description':
        bc_PCoA.plot(clMetadataNoNaN, column=col, axes=(0, 1, 2), axis_labels=['PC1','PC2','PC3'],title='samples colored by '+col+ ' BrayCurtis',cmap='viridis', s=20)
        plt.show()

In [ ]:
#jaccard PCoA
for col in clMetadataNoNaN.columns:
    if col != 'Description':
        j_PCoA.plot(clMetadataNoNaN, column=col, axes=(0, 1, 2), axis_labels=['PC1','PC2','PC3'],title='samples colored by '+col+ ' Jaccard',cmap='viridis', s=20)
        plt.show()

In [ ]:
plt.close()